In [1]:
import os

In [3]:
os.environ["HOST"] = "localhost"
os.environ["BBUY_DATA"] = "/Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products"
os.environ["WEEK1_PATH"] = "/Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/search_engineering/week1/"
os.environ["WEEK2_PATH"] = "/Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/search_engineering/week2/"


In [5]:
!curl -k -X PUT -u admin:admin "https://localhost:9200/bbuy_products" -H 'Content-Type: application/json' -d @$WEEK1_PATH/bbuy_products.json

{"acknowledged":true,"shards_acknowledged":true,"index":"bbuy_products"}

In [8]:
!python index.py -s $BBUY_DATA -w 16 -b 500

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 200000 and 500 per batch.
^C

Caught SIGINT. Shutting down workers...



Indexed 675577 documents (~50%) in 8m 37.1s. Introducing -m (as suggested by Dave) to limit number of files to spend less time indexing.

In [12]:
# clean up first
!curl -k -X DELETE -u admin:admin https://localhost:9200/bbuy_products


{"acknowledged":true}

In [13]:
!curl -k -X PUT -u admin:admin "https://localhost:9200/bbuy_products" -H 'Content-Type: application/json' -d @$WEEK1_PATH/bbuy_products.json

{"acknowledged":true,"shards_acknowledged":true,"index":"bbuy_products"}

In [14]:
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.909982067916667 minutes.  Total accumulated time spent in `bulk` indexing: 35.346926625566674 minutes


![Documents Indexed Rate](documents_indexed_rate_l1.png)

Re-indexing

In [15]:
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.821483982833333 minutes.  Total accumulated time spent in `bulk` indexing: 34.966568804966656 minutes


In [17]:
os.environ["BBUY_QUERIES"] = "/Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/"

In [ ]:
!python query.py -q $BBUY_QUERIES/train.csv -w 4 -m 25000


```bash
# truncated output

INFO:Loading query file from /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets//train.csv and using seed 84 for worker: 1
INFO:Loading query file from /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets//train.csv and using seed 126 for worker: 2
INFO:Loading query file from /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets//train.csv and using seed 168 for worker: 3
INFO:Loading query file from /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets//train.csv and using seed 42 for worker: 0
INFO:WN: 0: Running queries, checking in every 1000 queries:
INFO:WN: 1: Running queries, checking in every 1000 queries:
INFO:WN: 2: Running queries, checking in every 1000 queries:
INFO:WN: 3: Running queries, checking in every 1000 queries:

# ...

^C

Caught SIGINT. Shutting down workers...

WARNING:WN: 0: Failed to process query: Switchfoot
WARNING:WN: 3: Failed to process query: OnlineMidnightSale_Car_GPS
WARNING:WN: 1: Failed to process query: 2398896 2402035 5386263 5386272 6804112 8579932 8589878 9374278 9650424
WARNING:WN: 2: Failed to process query: car system
INFO:WN: 0: Stopped early.
INFO:WN: 3: Stopped early.
INFO:WN: 1: Stopped early.
INFO:WN: 2: Stopped early.
INFO:Query worker finished in time: 8.79507190305
INFO:Query worker finished in time: 8.795107660833335
INFO:Query worker finished in time: 8.795038516983334
INFO:Query worker finished in time: 8.795067490816667
```

Interrupted above after 8 minutes - observed similar bahaviour as in project meterials.
![Search Query Rate](search_query_rate_l1.png)
![Thread Pools](thread_pools_and_the_jvm_1.png)


## Level 2

### CPU 1 -> 2

In [20]:
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.7120400490833334 minutes.  Total accumulated time spent in `bulk` indexing: 22.36851046428335 minutes


A bit quicker, we're going above 1.5K, but not much.
![documents_indexed_rate_l2_cpu2.png](documents_indexed_rate_l2_cpu2.png)

In [ ]:
!python query.py -q $BBUY_QUERIES/train.csv -w 4 -m 25000

In [ ]:
# Truncated

Intersting results. Considerable boost to performance - almost 3x QPS with 3 threads active.

![QPS L2 2 CPU](search_query_rate_l2_cpu2.png)
![Threads L2 2 CPU](thread_pools_and_the_jvm_l2_cpu2.png)

### CPU 2 -> 4

In [22]:
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.3615369848666665 minutes.  Total accumulated time spent in `bulk` indexing: 19.266531507616673 minutes


Once again quicker, but not groundbreaking. 2k DPS more easily achieved.

![DPS L2 CPU 4](dps_l2_cpu4.png)

In [ ]:
!python query.py -q $BBUY_QUERIES/train.csv -w 4 -m 25000

In [24]:
# Truncated

A bit of improvement, we're going above 200 QPS. Overall it's not that dramatic as in CPU 1 -> CPU 2 case. It seems that GC is somehow blocking whole thread pool (4 threads) utilization.

![QPS L2 CPU 4](qps_l2_cpu4.png)
![Thread L2 CPU 4](threads_l2_cpu4.png)

### CPU 4 -> CPU 1, MEM 2 GB -> MEM 4 GB

Odd, but restarting opensearch-node1 wasn't enough to actually change Xms/Xmx. I had to stop everything in docker desktop, start it once again and re-install prometheus-exported plugin

In [25]:
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.99769373565 minutes.  Total accumulated time spent in `bulk` indexing: 35.26300945466666 minutes


The worst run so far. CPU is probably the limiting factor here. Thus we won't see much gains from adding more memory on 1 CPU.

![DPS L2 MEM 4](dps_l2_mem4.png)



In [ ]:
!python query.py -q $BBUY_QUERIES/train.csv -w 4 -m 25000

In [27]:
# Truncated

A bit unexpected - but makes sense. We did get a performance boost (roughly 2x QPS), due to better threadpool utilization - GC pauses do not affect search threads. So far (6 minutes) no GC seconds - visible on metrics.

![QPS L2 MEM 4](qps_l2_mem4.png)
![Threads L2 MEM 4](thread_l2_mem4.png)

### CPU 1 -> CPU 2, MEM 4
I assume we won't get much more gain from increasing only memory. As in case of DPS we've already hit the limit. We could get a bit more QPS with higher memory (we would probably have a constant search thread pool size of 2) but i'm geussing it's not worth it. Thus we're going with more CPU

In [29]:
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.3288495466 minutes.  Total accumulated time spent in `bulk` indexing: 8.634466537999998 minutes


The quickest so far, (previouse record - 4CPU)

![DPS L2 CPU 2, MEM 4](dps_l2_cpu2_mem4.png)

In [ ]:
!python query.py -q $BBUY_QUERIES/train.csv -w 4 -m 25000

In [31]:
# truncated

Best so far. We're reaching 250 QPS easily.

![QPS L2 CPU 2, MEM 4](qps_l2_cpu2_mem4.png)
![Threads L2 CPU 2, MEM 4](threads_l2_cpu2_mem4.png)

### CPU 4, MEM 4
Let's add more CPU

In [32]:
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.371222411416667 minutes.  Total accumulated time spent in `bulk` indexing: 9.2627948028 minutes


Not much difference here. Skipping the graph.

In [ ]:
!python query.py -q $BBUY_QUERIES/train.csv -w 4 -m 25000

In [34]:
# truncated

Interesting but we're not able to go above 250 QPS, CPU utilization is around 40%. What would be the limiting factor here?

![QPS L2 CPU 4, MEM 4](qps_l2_cpu4_mem4.png)
![Thread L2 CPU 4, MEM 4](thread_l2_cpu4_mem4.png)

### CPU 4, MEM 8

In [36]:
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.287822636583334 minutes.  Total accumulated time spent in `bulk` indexing: 7.345726475716666 minutes


The quickest one so far. But the change is pretty small (2.28 vs 2.32 CPU 2, MEM 4)

In [ ]:
!python query.py -q $BBUY_QUERIES/train.csv -w 4 -m 25000

And we have a winner. We've reached 300 QPS. So we had to double the RAM and CPUs to add 50 QPS.

![QPS L2 CPU 4, MEM 4](qps_l2_cpu4_mem8.png)
![Threads L2 CPU 4, MEM 4](thread_l2_cpu4_mem8.png)

## Level 3
Going with the last config

In [39]:
# running query.py in a separate shell

In [ ]:
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

In [40]:
# starting indexing once we have 300 QPS
!python index.py -s $BBUY_DATA -w 16 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 16 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.9295400737333335 minutes.  Total accumulated time spent in `bulk` indexing: 14.623565470449996 minutes


QPS dropped to ~50 during indexing. Indexing time did not change that dramatically, DPS ~1.5K.

![QPS L3](qps_l3.png)
![Threads L3](threads_l3.png)
![DPS L3](dps_l3.png)